<a href="https://colab.research.google.com/github/DustinHaschke/AlgDes_Projekt/blob/main/_AlgDes_Projekt_Trivialansatz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Aufzeigen einer möglichen Implementierung des Trivialansatzes


#reisezeiten[a][b] von a->b aus reisezeiten_namen-Index
reisezeiten = [[0,3,6,2,4],[3,0,4,2,1],[7,4,0,8,2],[4,3,7,0,4],[4,2,2,5,0]]

reisezeiten_max = 0
for i in range(len(reisezeiten)):
    for j in range(len(reisezeiten)):
        if reisezeiten[i][j] > reisezeiten_max:
            reisezeiten_max = reisezeiten[i][j]
print("Reisezeiten_max:", reisezeiten_max)

# Anmerkung: Bezeichnung wurde für Daten angepasst:  Stadthalle = Cinestar...,
reisezeiten_namen =["Filmhaus", "InfinityDome", "Kolosseum", "Koki", "CineStar"]
#Eintragung der Zulässigen Extraüberschneidungsminuten
zulaessige_ueberschneidungsminuten = 1

# Dateneinträge der Form:   Titel; Wochentag; Uhrzeit; Dauer; Spielort; Bewertung

small101 = """Omnejd;wed;19:00;42;InfinityDome;7.1
VR Expanded;wed;20:30;84;InfinityDome;7.7
Dancing Queen;thu;10:15;90;CineStar 1;7.5
VR Expanded;thu;17:00;84;InfinityDome;7.7
Omnejd;thu;18:45;42;InfinityDome;7.1
Overview;fri;11:15;32;InfinityDome;8.7
Opus 136;fri;11:47;10;InfinityDome;6.3
VR Expanded;fri;18:15;84;InfinityDome;7.7
Non-Face;fri;21:00;45;InfinityDome;6.5
Shorts 4+: Spiel Kino;sat;10:00;14;Koki;7.7
VR Expanded;sat;16:15;84;InfinityDome;7.7
Sigade revolutsioon | Revolution of Pigs | Revolution der Schweine;sat;22:15;98;CineStar 4;6.6
Opus 136;sun;11:44;10;InfinityDome;6.3
Omnejd;sun;14:30;43;InfinityDome;7.1
VR Expanded;sun;18:15;84;InfinityDome;7.7"""


# Festlegen des Datensatzes
datenliste = small101

# Datenvorbearbeitung inkl. Sortieren nach Uhrzeit Anfügen von Zeitminuten und der FilmID
def daten_vorbearbeitung(liste):
    # splitten der Zeileneintraege
    liste = liste.split("\n")

    # Verarbeitung als Liste
    for i in range(len(liste)):
        liste[i] = liste[i].split(";")

    # Anfügen an liste der "Gesamtminuten bis zum Beginn" (Wochentag+Uhrzeit)
    for i in range(len(liste)):
        if liste[i][1] == "mon":
            weekmin = 0
        elif liste[i][1] == "tue":
            weekmin = 1*60*24
        elif liste[i][1] == "wed":
            weekmin = 2*60*24
        elif liste[i][1] == "thu":
            weekmin = 3*60*24
        elif liste[i][1] == "fri":
            weekmin = 4*60*24
        elif liste[i][1] == "sat":
            weekmin = 5*60*24
        elif liste[i][1] == "sun":
            weekmin = 6*60*24
        else:
            weekmin = "Wochentag-Error"

        hourmin, basemin = liste[i][2].split(":")
        liste[i].append(int(weekmin) + int(hourmin)*60 + int(basemin))

    # Datenliste wird sortiert nach Universalminutenwert (über Standard-Python "Timsort")
    liste = sorted(liste, key=lambda x: x[6])

    # Eindeutige Film-ID anfügen der Form [Titel:[Gesamtminutenstart,Dauer,Kino]]
    film_id = {}
    for i in range(len(liste)):
        if liste[i][0] not in film_id:
              film_id[liste[i][0]] = []
        # Key Filmtitel wird mit [Gesamtminutenstart + Dauer + Kino] befüllt
        #film_id[liste[i][0]].append([liste[i][6], liste[i][3], liste[i][4]])                                            #toDOOOOOOOOOOOOOOOOOOOOOOOOOOOO geändert!!!!!
        film_id[liste[i][0]].append(i)

    #Ausgabe von liste als zeitlich sortierter Spielplan in Form [Titel, Wochentag, Uhrzeit, Dauer, Spielort, Bewertung, StartzeitGesamtminuten]
    #Ausgabe von film_id in Form {Titel:[Gesamtminutenstart,Dauer,Kino]}
    return liste, film_id


# Überschneidungsgruppen werden mit Zuordnung über den Indexwert gebildet
#           ANMERKUNG: theoretische Laufzeit n+n-1+n-2...+1 ~ n², aber faktisch überschneiden sich kaum Titel, wodurch die Laufzeit eher in n liegt!
def ueberschneidungs_gruppen(zeitsortierte_daten):
    # hierfür werden alle möglichen Überschneidungen für den Film mit Index i aus zeitsortierteDaten[i] in ueberschneidungsliste[i] eingetragen
    ueberschneidungsliste = []
    # Anlegen der notwendigen Listen von Listen
    for i in range(len(zeitsortierte_daten)):
        ueberschneidungsliste.append([])

    for i in range(len(zeitsortierte_daten)-1):
        abstand = 1
        fertig_gruppiert = False

        # eigenen Indexwert eintragen um später Duplikate zu entfernen
        ueberschneidungsliste[i].append(i)

        while(not(fertig_gruppiert)):

            # Reisezeit von Spielort von i nach i+abstand ergibt sich aus oberer Matrix
            # Hinweis: Da einige Spielortdaten eine Zahl enthielten nach einem vorherigen Leerzeichen, wird diese abgetrennt!
            a = zeitsortierte_daten[i][4].split(" ")[0]
            b = zeitsortierte_daten[i+abstand][4].split(" ")[0]
            reisezeit =  reisezeiten[reisezeiten_namen.index(a)][reisezeiten_namen.index(b)]
            # zulässige ankunftszeit = StartzeitGesamtminuten + Dauer - zulaessige_ueberschneidungsminuten + Reisezeit
            ankunftszeit = zeitsortierte_daten[i][6] + int(zeitsortierte_daten[i][3]) - zulaessige_ueberschneidungsminuten + reisezeit

            #Falls Ankunftszeit zu spät für aktuelle folgende Vorstellung über "abstand"
            if ankunftszeit > zeitsortierte_daten[i+abstand][6]:
                # ...trage in ueberschneidungsliste[i] überschneidende Vorstellungsindex ein
                ueberschneidungsliste[i].append(i+abstand)
                # ... und trage in ueberschneidungsliste[i+abstand] den i-ten Vorstellungsindex ein
                ueberschneidungsliste[i+abstand].append(i)

                #Abstand erhöht sich und nächster Eintrag wird getestet
                abstand += 1
            #ansonsten keine Überschneidungen mehr mit Vorstellung
            else:
                fertig_gruppiert = True

            # keine Überschneidungen mehr, falls abstand+i letzter Eintrag von Daten
            if abstand+i ==len(zeitsortierte_daten):
                fertig_gruppiert = True

    # letzte eigenen Indexwert eintragen um später Duplikate zu entfernen
    ueberschneidungsliste[len(zeitsortierte_daten)-1].append(len(zeitsortierte_daten)-1)

    # gebe Liste ohne Duplikate zurück
    ueberschneidungen_ohne_duplikate = []
    for element in ueberschneidungsliste:
        if element not in ueberschneidungen_ohne_duplikate:
            ueberschneidungen_ohne_duplikate.append(element)

    print("ueberschneidungen_ohne_duplikate:", ueberschneidungen_ohne_duplikate)
    return ueberschneidungen_ohne_duplikate

# bei einer trivialen Liste ohne Überschneidungen liefert diese unmittelbar die Filmliste
def entferne_ohneUeberschneidungen(zeitsortierte_datenliste, rest_ueberschneidungen, film_id):
    gesicherte_filmzeiten = []
    still_removing = True
    while still_removing:
        still_removing = False
        for i in range(len(rest_ueberschneidungen)):
            # Falls Eintrag keine Ueberschneidungen mit anderen Filmen hat:
            if rest_ueberschneidungen[i] == [i]:
                # Rufe Titel von Datenliste ab und speichere in gesicherte Filmzeiten
                titel = zeitsortierte_datenliste[i][0]
                gesicherte_filmzeiten.append(zeitsortierte_datenliste[i])                                           # toDOOOOOOOOOOOOOOO!!!!

                # Entferne alle Filme gleichen Titels aus Ueberschneidungen ÜBER FilmID
                for index in film_id[titel]:
                    for filmgruppe in rest_ueberschneidungen:
                        if index in filmgruppe:
                            filmgruppe.remove(index)
                # Falls noch Änderungen vorgenommen wurden, so wiederhole!
                still_removing = True

        return(rest_ueberschneidungen, gesicherte_filmzeiten)

def zaehle_ranking(gesicherte_filme):
    imdb_score = 0
    for element in gesicherte_filme:
        imdb_score += float(element[5])
    return imdb_score


def reduziere_um_FilmID(gewaehlter_index, zeitsortierte_datenliste, ueberschneidungen, film_id):
    name = zeitsortierte_datenliste[gewaehlter_index]


zeitsortierte_datenliste, film_id = daten_vorbearbeitung(datenliste)
ueberschneidungen = ueberschneidungs_gruppen(zeitsortierte_datenliste)
print("zeitsortierte Daten:", zeitsortierte_datenliste)
print("film_id:", film_id)
print("Überschneidungen:", ueberschneidungen)
neue_ueberschneidungen, gesicherte_filmzeiten = entferne_ohneUeberschneidungen(zeitsortierte_datenliste, ueberschneidungen, film_id)

print("neue_ueberschneidungen:", neue_ueberschneidungen)
print("gesicherte_filmzeiten:", gesicherte_filmzeiten)

# Prüfe, ob Trivialansatz komplett reduziert:
if len(gesicherte_filmzeiten) == len(film_id):
    max_score = zaehle_ranking(gesicherte_filmzeiten)
    print("Maximaler Score für Datensatz:", max_score)

else:
    print("Ansonsten: z.B. Branch-and-Bound-Ansatz")


Reisezeiten_max: 8
ueberschneidungen_ohne_duplikate: [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14]]
zeitsortierte Daten: [['Omnejd', 'wed', '19:00', '42', 'InfinityDome', '7.1', 4020], ['VR Expanded', 'wed', '20:30', '84', 'InfinityDome', '7.7', 4110], ['Dancing Queen', 'thu', '10:15', '90', 'CineStar 1', '7.5', 4935], ['VR Expanded', 'thu', '17:00', '84', 'InfinityDome', '7.7', 5340], ['Omnejd', 'thu', '18:45', '42', 'InfinityDome', '7.1', 5445], ['Overview', 'fri', '11:15', '32', 'InfinityDome', '8.7', 6435], ['Opus 136', 'fri', '11:47', '10', 'InfinityDome', '6.3', 6467], ['VR Expanded', 'fri', '18:15', '84', 'InfinityDome', '7.7', 6855], ['Non-Face', 'fri', '21:00', '45', 'InfinityDome', '6.5', 7020], ['Shorts 4+: Spiel Kino', 'sat', '10:00', '14', 'Koki', '7.7', 7800], ['VR Expanded', 'sat', '16:15', '84', 'InfinityDome', '7.7', 8175], ['Sigade revolutsioon | Revolution of Pigs | Revolution der Schweine', 'sat', '22:15', '98', 'CineStar 4', '6.6',